In [ ]:
import numpy as np
import pandas as pd

## Dataset:

In [177]:
colnames = ['eng', 'ni1', 'ni2', 'vn']
path = '/content/drive/MyDrive/Fall 2022 - Assignment/Software Construction/Project/full_data/final_df.csv'
df = pd.read_csv(path, header=0)
print(df.shape)

(39034, 5)


In [178]:
df.head(4)

,Unnamed: 0,eng,ni1,ni2,vn
0,0,arthropathology (n),"[,@:8r6up6’80l6d2i]",k,bệnh học khớp
1,1,-arthropathy (n),[@:’8r0p68i],arthropathie (f),bệnh khớp
2,2,arthrophyma (n),"[,@:8r6u’faim6]",k,sưng khớp
3,3,arthrophyte (n),[‘@:8r6fait],arthrophyte (m),dị vật khớp


In [179]:
df = df.drop('Unnamed: 0', axis=1)

In [180]:
df = df.drop(['ni1', 'ni2'], axis=1)

### Droping NAs

In [181]:
missing = df[df.isna().any(axis=1)]
#missing.to_csv('/content/drive/MyDrive/Fall 2022 - Assignment/Software Construction/Project/full_data/final_missing.csv')
missing.shape 

(2679, 2)

In [182]:
# Dropping NAs
df = df.dropna(axis='index', how='any')
print(df.shape)

(36355, 2)


### Split cases with ```/``` into multiple rows
Example: ```abulia (n) / abulic (a)	``` becomes 2 rows ```abulia (n)``` and ```abulic (a)```

In [183]:
df_split = df.loc[df.eng.str.contains('/'), :].copy() # create a new df of only rows with '/'

df = df.loc[~df.eng.str.contains('/'), :].copy() # remove them for now

print(df_split.shape)
print(df.shape)

(1598, 2)
(34757, 2)


In [184]:
# SOURE: https://stackoverflow.com/questions/58523316/split-rows-in-pandas-dataframe

# for index, row in df.iterrows():
#     split = row['Shape'].split(' / ')
#     for shape in split:
#         new_df = new_df.append(pd.DataFrame({'Color':[row['Color']], 'Shape':[shape], 'Price':[row['Price']]}))

# new_df = new_df.reset_index(drop=True)

df_split['eng'] = df_split.eng.str.split(" / ")
df_split = df_split.explode("eng").reset_index(drop=True)
df_split.head(2)
print(df_split.shape)

(3181, 2)


In [185]:
# Append the splitted '/' cases back to the orginal data

df = df.append(df_split, ignore_index=True)
df.shape

(37938, 2)

### Split cases with ```,``` into multiple rows
Example: ```abiogenetic, abiogenous (a)``` becomes 2 rows : ```abiogenetic (a)``` and ```abiogenous (a)```

In [186]:
df_split_comma = df.loc[df.eng.str.contains(', '), :].copy() # extract rows with ', '
df = df.loc[~df.eng.str.contains(', '), :].copy() # temporarily remove these rows
print(df.shape)
print(df_split_comma.shape)

(36601, 2)
(1337, 2)


In [187]:
df_split_comma['eng'] = df_split_comma.eng.str.split(", ")
df_split_comma = df_split_comma.explode("eng").reset_index(drop=True)
df_split_comma.head(2)

,eng,vn
0,arthropneumography,chụp khớp bơm khí
1,arthropneumoradiography (n),chụp khớp bơm khí


**Note**: Later on when adding word types (a, n, v...) we will use ```fillna('bfill')``` to account for those without a type
Example: ```-aberration``` comes right before ```aberratio (n)``` so will be backfilled with ```(n)```

In [188]:
df = df.append(df_split_comma, ignore_index=True)
print(df.shape)

(39347, 2)


### Extract word types (n a adj v...)

In [189]:
df['word_type'] = df.eng
df.head(3)

,eng,vn,word_type
0,arthropathology (n),bệnh học khớp,arthropathology (n)
1,-arthropathy (n),bệnh khớp,-arthropathy (n)
2,arthrophyma (n),sưng khớp,arthrophyma (n)


#### Remove cases with special ```ñn```

In [190]:
def remove_special(entry):
  '''
  For words that contain 'ñn', remove the part that comes after this symbol
  '''
  if 'ñn' in entry:
    return entry.split('ñn')[0]
  else:
    return entry
# lambda x: x.split('ñn')[0] if 'ñn' in x else x

#### Extract word types

In [191]:
def get_word_type(entry):
  '''
  Works for easy cases. Extract the word type (n, a, v, ...) inside the brackets '()'
  '''
  entry = entry.rstrip()
  if '(' in entry:
    if len(entry.split('(')) == 2:
      return entry.split('(')[1][:-1]

    else:
      return "More than 1 ()'s"

  else:
    return np.nan

In [192]:
df.word_type = df.word_type.apply(remove_special)
df.word_type = df.word_type.apply(get_word_type)

In [193]:
df_word_type_na = df.loc[df.word_type.isna(), :].copy() # extract all words with NA
df = df.loc[~df.word_type.isna(), :].copy() # temporarily remove these words

print(df.shape)
print(df_word_type_na.shape)

(37725, 3)
(1622, 3)


#### Fill NAs
Before, there were a few rows resulted from splitting ```', '``` that don't have a word type ```(n)``` afterwards. We fix this by matching them with the word type of the **"true"** words - which are the final words in a ```', '``` that contain the word type.

Example: ```-aberration, aberratio (n)``` => ```abrratio``` is the **true** word, containing the type ```(n)```.

In [194]:
def get_word_type_for_na(entry):
  if entry in df_split_comma.eng.values:
    curr_idx = np.where(df_split_comma.eng.values == entry)[0][0] # index of the entry

    move = 1
    while df_split_comma.eng.values[curr_idx + move][-1] != ')': # sometimes more than 1 word before "true" word
      move += 1 # increase this until we find the "true" label.

    type_idx = curr_idx + move # "true" word with word type
    type_value = df_split_comma.eng.values[type_idx]

    try:
      return  df.loc[df.eng==type_value, 'word_type'].values[0]
    except IndexError:
      return np.nan
  
  else:
    return np.nan

In [195]:
df_word_type_na.word_type = df_word_type_na.eng
df_word_type_na.word_type = df_word_type_na.word_type.apply(get_word_type_for_na)
df_word_type_na.tail(10)

,eng,vn,word_type
39327,brachycephaly,ngắn đầu (tật),n
39329,brachycephalic,ngắn đầu (tật),a
39331,bronchiectasic,giãn phế quản (ch),a
39333,-bronchopneumonia,viêm phế quản-phổi,n
39335,-ambidexterity,thuận hai tay,n
39337,-amyotrophy,teo cơ (ch),n
39339,anachoretic,dồn tới một nơi (sự),a
39341,anagogy,giải thích thần bí,n
39343,stenocephaly,đầu hẹp,n
39345,stromal,"chất đệm, chất nền,mô đỡ",a


In [196]:
df = df.append(df_word_type_na, ignore_index=True)
df.shape

(39347, 3)

### Too edgy edge cases
The rest are edge cases with no rules or standards for now.

In [197]:
df.word_type.value_counts()

n                        27005
a                         9592
More than 1 ()'s           612
a,n                        485
v                          332
                         ...  
median lethal dose           1
pref.)  copr                 1
pref)  colp                  1
kilogram                     1
a)///////////////////        1
Name: word_type, Length: 177, dtype: int64

In [198]:
df[df.word_type=='diffraction']

,eng,vn,word_type
8787,grating (diffraction),mạng nhiễu xạ,diffraction


In [199]:
edge = df_word_type_na.loc[df_word_type_na.word_type.isna(), :].copy()
df_word_type_na = df_word_type_na.loc[~df_word_type_na.word_type.isna(), :].copy()

df = df.append(df_word_type_na, ignore_index=True)

In [200]:
more_edge = df.loc[df.word_type=="More than 1 ()'s", :].copy()
df = df.loc[df.word_type!="More than 1 ()'s", :].copy()

edge = edge.append(more_edge, ignore_index=True)

In [201]:
edge

,eng,vn,word_type
0,-articulo-mortis,"lúc hấp hối, lúc lâm chung",NaN
1,-ascorbic acid,acid ascorbic,NaN
2,-atrioventricularis communis,ống nhĩ thất chung,NaN
3,-axis cylinder,"sợi trục, trụ trục",NaN
4,anat.,giải phẫu học,NaN
...,...,...,...
873,-babesiasis,bệnh nhiễm Babesia,More than 1 ()'s
874,bacilli-,trực khuẩn,More than 1 ()'s
875,baritosis,bệnh bụi bari phổi,More than 1 ()'s
876,sarsaparilla,thổ phục linh,More than 1 ()'s


In [202]:
# RUN ONCE
edge.to_csv('/content/drive/MyDrive/Fall 2022 - Assignment/Software Construction/Project/full_data/edge_final.csv')

#### Remove the word types in ```eng``` column

In [203]:
df.head()

,eng,vn,word_type
0,arthropathology (n),bệnh học khớp,n
1,-arthropathy (n),bệnh khớp,n
2,arthrophyma (n),sưng khớp,n
3,arthrophyte (n),dị vật khớp,n
4,arthropod (n),"chân khớp (động vật), tiết chi động vật",n


In [204]:
# Idea is to split on "(", and keep only the [0] part
def remove_word_type(entry):
  if "(" in entry:
    keep = entry.split('(')[0].rstrip()
    return keep
  else:
    return entry

df.eng = df.eng.apply(remove_word_type)

### Add new ```word_type_vn``` column


In [205]:
df.word_type.value_counts()

n                        27873
a                        10032
a,n                        499
v                          338
pref.                      171
                         ...  
median lethal dose           1
pref.)  copr                 1
pref)  colp                  1
kilogram                     1
a)///////////////////        1
Name: word_type, Length: 176, dtype: int64

In [206]:
df['word_type_vn'] = df.word_type


In [207]:
df.word_type_vn.value_counts()

n                        27873
a                        10032
a,n                        499
v                          338
pref.                      171
                         ...  
median lethal dose           1
pref.)  copr                 1
pref)  colp                  1
kilogram                     1
a)///////////////////        1
Name: word_type_vn, Length: 176, dtype: int64

In [209]:
df.word_type_vn.value_counts().head(45)

n                             27873
a                             10032
a,n                             499
v                               338
pref.                           171
a.n                             116
F                                87
suff.                            86
L                                71
m                                41
ad                               36
 n                               35
pref.)                           34
f                                27
o)                               22
v,n                              20
disorder                         19
n,v                              17
pref.)                           16
ad.                              16
suff.)                           15
pref                             15
                                 14
suff.)                           12
n)                                8
v.n                               6
pref)                             6
a,v                         

In [210]:
def get_word_type_vn(entry):
  entry = str(entry)
  if entry == 'n':
    return 'danh từ'
  if entry == 'a':
    return 'tính từ'
  if entry == 'v':
    return 'động từ'
  if 'pref' in entry: # prefix
    return 'tiền tố'
  if 'suff' in entry:
    return 'hậu tố'
  else:
    if 'a' in entry and len(entry) <= 3:
      return 'tính từ'
    if 'n' in entry and len(entry) <= 3:
      return 'danh từ'
    if 'v' in entry and len(entry) <= 3:
      return 'động từ'
  return entry

In [211]:
df.head()

,eng,vn,word_type,word_type_vn
0,arthropathology,bệnh học khớp,n,n
1,-arthropathy,bệnh khớp,n,n
2,arthrophyma,sưng khớp,n,n
3,arthrophyte,dị vật khớp,n,n
4,arthropod,"chân khớp (động vật), tiết chi động vật",n,n


In [212]:
df.word_type_vn = df.word_type_vn.apply(get_word_type_vn)

In [213]:
df.head(10)

,eng,vn,word_type,word_type_vn
0,arthropathology,bệnh học khớp,n,danh từ
1,-arthropathy,bệnh khớp,n,danh từ
2,arthrophyma,sưng khớp,n,danh từ
3,arthrophyte,dị vật khớp,n,danh từ
4,arthropod,"chân khớp (động vật), tiết chi động vật",n,danh từ
5,-Arthropoda,chân khớp (ngành),n,danh từ
6,-arthropyosis,sinh mủ khớp (sự),n,danh từ
7,arthrorisis,hạn chế khớp (thth),n,danh từ
8,arthroscintigram,nhấp nháy đồ khớp,n,danh từ
9,arthroscintigraphy,chụp nhấp nháy khớp (thuật),n,danh từ


### Remove ```'-'``` before words

In [214]:
df.eng = df.eng.apply(lambda x: x.lstrip('-'))

### Final version

In [215]:
df.head(4)

,eng,vn,word_type,word_type_vn
0,arthropathology,bệnh học khớp,n,danh từ
1,arthropathy,bệnh khớp,n,danh từ
2,arthrophyma,sưng khớp,n,danh từ
3,arthrophyte,dị vật khớp,n,danh từ


In [216]:
df.isna().any()

eng             False
vn              False
word_type        True
word_type_vn    False
dtype: bool

In [227]:
na_word_type = df.loc[df.word_type.isna(), :].copy()
df = df.loc[~df.word_type.isna(), :].copy()

In [228]:
df.shape

(39841, 4)

In [229]:
# RUN ONCE

df.to_csv('/content/drive/MyDrive/Fall 2022 - Assignment/Software Construction/Project/full_data/final_df.csv')
na_word_type.to_csv('/content/drive/MyDrive/Fall 2022 - Assignment/Software Construction/Project/full_data/na_word_type.csv')

In [230]:
df

,eng,vn,word_type,word_type_vn
0,arthropathology,bệnh học khớp,n,danh từ
1,arthropathy,bệnh khớp,n,danh từ
2,arthrophyma,sưng khớp,n,danh từ
3,arthrophyte,dị vật khớp,n,danh từ
4,arthropod,"chân khớp (động vật), tiết chi động vật",n,danh từ
...,...,...,...,...
40714,amyotrophy,teo cơ (ch),n,danh từ
40715,anachoretic,dồn tới một nơi (sự),a,tính từ
40716,anagogy,giải thích thần bí,n,danh từ
40717,stenocephaly,đầu hẹp,n,danh từ
